<a href="https://colab.research.google.com/github/vikysena9-design/python-ai-Rudnikova-Viktoria/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Week 2: Data Analysis — Чтение и проверка данных

**Цель**: Научиться читать CSV-файлы из репозитория GitHub в Google Colab и выполнять базовую проверку данных с помощью pandas.

**Данные:**
- `horses.csv` — информация о лошадях: клички, годы жизни, пол, порода

**Что мы делаем:**
1. Клонируем ваш репозиторий GitHub в Colab
2. Читаем CSV-файл с данными о лошадях в pandas DataFrame
3. Очищаем и анализируем структуру данных
4. Выполняем быструю валидацию: проверяем количество записей, типы данных, пропуски

## 🐱 [1] Клонируем репозиторий курса в Colab

In [ ]:
# 🐴 Шаг 1. Клонируем ваш репозиторий в Colab

import os

if not os.path.exists("python-ai-Rudnikova-Viktoria"):
    !git clone -q https://github.com/vikysena9-design/python-ai-Rudnikova-Viktoria.git

%cd python-ai-Rudnikova-Viktoria

print("✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Rudnikova-Viktoria")

/content/python-ai-Rudnikova-Viktoria/python-ai-Rudnikova-Viktoria
✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Rudnikova-Viktoria


## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [ ]:
# 🐴 Шаг 2. Чтение CSV-файла с данными о лошадях

import pandas as pd

df_horses = pd.read_csv("data/horses.csv")

print("✅ Загружено строк в датасет:", len(df_horses))
print("\n📋 Первые 3 строки данных:")
print(df_horses.head(3))
print("\n🔍 Структура данных:")
print(df_horses.info())

✅ Загружено строк в датасет: 12309

📋 Первые 3 строки данных:
                                     horse        horseLabel  birthYear  \
0  http://www.wikidata.org/entity/Q5172152  Coronado's Quest     1995.0   
1  http://www.wikidata.org/entity/Q5180163           Cozzene     1980.0   
2  http://www.wikidata.org/entity/Q5190874   Cryptoclearance     1984.0   

   deathYear genderLabel             breedLabel  
0     2006.0       самец  чистокровная верховая  
1     2008.0       самец  чистокровная верховая  
2     2009.0       самец  чистокровная верховая  

🔍 Структура данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12309 entries, 0 to 12308
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   horse        12309 non-null  object 
 1   horseLabel   12309 non-null  object 
 2   birthYear    12308 non-null  float64
 3   deathYear    1581 non-null   float64
 4   genderLabel  11280 non-null  object 
 5   breedLabe

## 🧹 [2B] Очистка и переименование столбцов

В исходном CSV-файле есть **технические столбцы**, которые полезны для Викиданных, но мешают простому анализу:

- Столбец `horse` с URL (ссылкой на объект Wikidata) — нам не нужна ссылка, нам достаточно клички лошади.
- Столбцы `horseLabel`, `genderLabel`, `breedLabel` содержат читаемые подписи (кличка, пол, порода).

В этом шаге мы:
- удалим столбец с URL Wikidata (`horse`);
- переименуем `horseLabel → horse`, `genderLabel → gender`, `breedLabel → breed`;
- приведём числовые столбцы (`birthYear`, `deathYear`) к типу `int`.

При приведении к числам мы используем:

- `pd.to_numeric(..., errors="coerce")` — преобразует значения в числа, некорректные значения превращает в `NaN`;
- `fillna(0)` — заменяет пропущенные значения (`NaN`) на 0;
- `astype(int)` — переводит столбец к целочисленному типу.

> ⚠️ **Важно:** если в ваших данных есть столбцы с URL Wikidata и столбцы вида `*Label`, этот шаг обязателен, чтобы получить аккуратные таблички для анализа.

In [11]:
# 🧹 Шаг 2B. Очистка и переименование столбцов (безопасная версия)

# Проверяем, есть ли исходные столбцы Wikidata — если да, выполняем очистку
if "horse" in df_horses.columns and "horseLabel" in df_horses.columns:
    # Удаляем только столбец с URL (технический идентификатор)
    df_horses = df_horses.drop(columns=["horse"])

    # Переименовываем *Label-столбцы только если они существуют
    rename_mapping = {}
    if "horseLabel" in df_horses.columns:
        rename_mapping["horseLabel"] = "horse"
    if "genderLabel" in df_horses.columns:
        rename_mapping["genderLabel"] = "gender"
    if "breedLabel" in df_horses.columns:
        rename_mapping["breedLabel"] = "breed"

    if rename_mapping:
        df_horses = df_horses.rename(columns=rename_mapping)

    # Приводим числовые столбцы к целочисленному типу
    for col in ["birthYear", "deathYear"]:
        if col in df_horses.columns:
            df_horses[col] = pd.to_numeric(
                df_horses[col], errors="coerce"
            ).fillna(0).astype(int)

    print("✅ Данные успешно очищены и подготовлены к анализу")

elif "horse" in df_horses.columns and "horseLabel" not in df_horses.columns:
    print("ℹ️ Данные уже очищены ранее — столбцы *Label отсутствуют")

else:
    print("⚠️ Неожиданная структура данных: проверьте наличие столбцов horse/horseLabel")

print("\n📋 Структура датасета после обработки:")
print(df_horses.info())
print("\n🔍 Пример данных:")
print(df_horses.head(3))

✅ Данные успешно очищены и подготовлены к анализу

📋 Структура датасета после обработки:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12309 entries, 0 to 12308
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   horse      12309 non-null  object
 1   birthYear  12309 non-null  int64 
 2   deathYear  12309 non-null  int64 
 3   gender     11280 non-null  object
 4   breed      12309 non-null  object
dtypes: int64(2), object(3)
memory usage: 480.9+ KB
None

🔍 Пример данных:
              horse  birthYear  deathYear gender                  breed
0  Coronado's Quest       1995       2006  самец  чистокровная верховая
1           Cozzene       1980       2008  самец  чистокровная верховая
2   Cryptoclearance       1984       2009  самец  чистокровная верховая


## 🔍 [3] Обзор данных: структура и первые строки

Сделаем короткий обзор датасета с лошадьми:

- посмотрим размер таблицы (`shape`);
- выведем список столбцов и их типы данных;
- посмотрим первые несколько строк для понимания содержимого;
- дополнительно посчитаем базовую статистику по годам жизни (`birthYear`, `deathYear`).

Для удобства напишем функцию `show_info(df, name)`, которая покажет всю ключевую информацию о датасете одной командой — это пригодится при работе с несколькими таблицами в будущем.

In [12]:
def show_info(df, name, n=5):
    """Краткий обзор DataFrame: имя, размер, типы столбцов и первые строки."""
    print(f"\n📊 {name}")
    print("Размер:", df.shape)
    print("\nСтолбцы и типы данных:")
    for col in df.columns:
        print(f"  • {col:20s} → {df[col].dtype}")
    print("\nПервые строки:")
    print(df.head(n))

# 🔍 Шаг 3. Обзор данных (устойчивый к состоянию очистки)

show_info(df_horses, "Данные о лошадях (df_horses)")

# Определяем актуальные имена столбцов в зависимости от состояния данных
col_birth = "birthYear" if "birthYear" in df_horses.columns else None
col_death = "deathYear" if "deathYear" in df_horses.columns else None
col_gender = "gender" if "gender" in df_horses.columns else ("genderLabel" if "genderLabel" in df_horses.columns else None)
col_breed = "breed" if "breed" in df_horses.columns else ("breedLabel" if "breedLabel" in df_horses.columns else None)

# Статистика по годам рождения
if col_birth:
    print(f"\n📈 Статистика по годам рождения ({col_birth}):")
    stats = df_horses[col_birth].describe()
    for stat, val in stats.items():
        print(f"  {stat:8s}: {int(val) if pd.notna(val) and val == int(val) else val}")

# Статистика по годам смерти
if col_death:
    print(f"\n📈 Статистика по годам смерти ({col_death}):")
    stats = df_horses[col_death].describe()
    for stat, val in stats.items():
        print(f"  {stat:8s}: {int(val) if pd.notna(val) and val == int(val) else val}")

# Распределение по полу
if col_gender:
    print(f"\n📊 Распределение по полу ({col_gender}):")
    print(df_horses[col_gender].value_counts(dropna=False))

# Топ-5 пород
if col_breed:
    print(f"\n🏆 Топ-5 пород ({col_breed}):")
    print(df_horses[col_breed].value_counts().head(5))


📊 Данные о лошадях (df_horses)
Размер: (12309, 5)

Столбцы и типы данных:
  • horse                → object
  • birthYear            → int64
  • deathYear            → int64
  • gender               → object
  • breed                → object

Первые строки:
              horse  birthYear  deathYear gender                  breed
0  Coronado's Quest       1995       2006  самец  чистокровная верховая
1           Cozzene       1980       2008  самец  чистокровная верховая
2   Cryptoclearance       1984       2009  самец  чистокровная верховая
3           Cyllene       1895       1925  самец  чистокровная верховая
4           Da Hoss       1992       2022    NaN  чистокровная верховая

📈 Статистика по годам рождения (birthYear):
  count   : 12309
  mean    : 1961.9411812494923
  std     : 59.5892119484096
  min     : 0
  25%     : 1941
  50%     : 1982
  75%     : 2001
  max     : 2024

📈 Статистика по годам смерти (deathYear):
  count   : 12309
  mean    : 255.95864814363475
  std     : 

## 🏆 Породы-долгожители и гендерная карта выживаемости

В этом анализе мы ответим на два ключевых вопроса о лошадях:

### 1. Какие породы живут дольше всего?
Мы рассчитаем **среднюю продолжительность жизни** для каждой породы, используя формулу:  
`возраст = год смерти − год рождения`

> ⚠️ **Важно:** анализ включает только лошадей с известным годом смерти (умершие особи). Лошади без данных о смерти исключаются, чтобы избежать искажения статистики.

### 2. Кто живёт дольше: жеребцы или кобылы?
Мы сравним среднюю продолжительность жизни **по полу внутри каждой породы**, чтобы выявить:
- Есть ли системная разница в выживаемости между самцами и самками
- В каких породах эта разница наиболее выражена (возможно, из-за спортивной нагрузки или условий содержания)

### 🔬 Методология очистки данных:
- Отфильтрованы аномалии: возраст < 0 лет (ошибки в данных) и > 70 лет (максимальный задокументированный возраст лошади — 62 года)
- Учтены только породы с достаточной выборкой (≥ 15 особей) для статистической надёжности
- Для гендерного анализа использованы только породы, где представлены оба пола

Этот анализ раскроет скрытые закономерности в биологии и истории коневодства — от «долгожительниц» среди рабочих пород до гендерного неравенства в спортивных дисциплинах.

In [14]:
# 🏆 Шаг 4. Анализ долголетия: породы и гендерные различия

import pandas as pd
import numpy as np

# ============================================================================
# 0. Подготовка данных: определение актуальных имён столбцов
# ============================================================================
col_horse = "horse" if "horse" in df_horses.columns and "horseLabel" not in df_horses.columns else "horseLabel"
col_birth = "birthYear"
col_death = "deathYear"
col_gender = "gender" if "gender" in df_horses.columns else "genderLabel"
col_breed = "breed" if "breed" in df_horses.columns else "breedLabel"

# Создаём копию для анализа
df_analysis = df_horses.copy()

# ============================================================================
# 1. Расчёт возраста и фильтрация аномалий
# ============================================================================
# Оставляем только лошадей с известными годами рождения и смерти
df_analysis = df_analysis[df_analysis[col_death].notna() & df_analysis[col_birth].notna()]

# Рассчитываем возраст
df_analysis["age"] = df_analysis[col_death] - df_analysis[col_birth]

# Фильтруем аномалии: возраст от 0 до 70 лет (реалистичный диапазон)
df_analysis = df_analysis[(df_analysis["age"] >= 0) & (df_analysis["age"] <= 70)]

print(f"✅ Для анализа отобрано {len(df_analysis)} лошадей с корректными данными о возрасте\n")

# ============================================================================
# 2. Породы-долгожители: рейтинг по средней продолжительности жизни
# ============================================================================
print("="*70)
print("🏆 ТОП-10 ПОРОД ПО СРЕДНЕЙ ПРОДОЛЖИТЕЛЬНОСТИ ЖИЗНИ")
print("="*70)

# Группируем по породе
breed_stats = df_analysis.groupby(col_breed).agg(
    avg_age=("age", "mean"),
    median_age=("age", "median"),
    count=("age", "count"),
    min_age=("age", "min"),
    max_age=("age", "max")
).round(2)

# Фильтруем породы с достаточной выборкой (≥15 особей)
breed_stats = breed_stats[breed_stats["count"] >= 15].sort_values("avg_age", ascending=False)

# Выводим топ-10
if not breed_stats.empty:
    for idx, (breed, row) in enumerate(breed_stats.head(10).iterrows(), 1):
        print(f"{idx:2d}. {breed:30s} | Средний: {row['avg_age']:4.1f} лет | "
              f"Медиана: {row['median_age']:4.1f} | "
              f"N={int(row['count']):3d} | Диапазон: {int(row['min_age'])}–{int(row['max_age'])} лет")
    print(f"\nℹ️  Всего проанализировано {len(breed_stats)} пород с выборкой ≥15 особей")
else:
    print("⚠️  Недостаточно данных для анализа пород (требуется ≥15 особей на породу)")

# ============================================================================
# 3. Гендерная карта выживаемости
# ============================================================================
print("\n" + "="*70)
print("⚧️  ГЕНДЕРНАЯ КАРТА ВЫЖИВАЕМОСТИ")
print("="*70)

# Проверяем наличие обоих полов в данных
unique_genders = df_analysis[col_gender].dropna().unique()
print(f"Доступные значения пола в данных: {list(unique_genders)}")

# Продолжаем анализ ТОЛЬКО если есть оба пола: "самец" и "самка"
if "самец" in unique_genders and "самка" in unique_genders:
    # Группируем по породе и полу
    gender_breed = df_analysis.groupby([col_breed, col_gender]).agg(
        avg_age=("age", "mean"),
        count=("age", "count")
    ).round(2).reset_index()

    # Оставляем только породы с обоими полами и достаточной выборкой (≥5 особей на пол)
    valid_breeds = []
    for breed in gender_breed[col_breed].unique():
        breed_data = gender_breed[gender_breed[col_breed] == breed]
        males = breed_data[breed_data[col_gender] == "самец"]
        females = breed_data[breed_data[col_gender] == "самка"]
        if not males.empty and not females.empty:
            if males.iloc[0]["count"] >= 5 and females.iloc[0]["count"] >= 5:
                valid_breeds.append(breed)

    if valid_breeds:
        # Строим сравнительную таблицу
        results = []
        for breed in valid_breeds:
            breed_data = gender_breed[gender_breed[col_breed] == breed]
            male_avg = breed_data[breed_data[col_gender] == "самец"].iloc[0]["avg_age"]
            female_avg = breed_data[breed_data[col_gender] == "самка"].iloc[0]["avg_age"]
            male_cnt = int(breed_data[breed_data[col_gender] == "самец"].iloc[0]["count"])
            female_cnt = int(breed_data[breed_data[col_gender] == "самка"].iloc[0]["count"])
            diff = female_avg - male_avg

            results.append({
                "breed": breed,
                "female_age": female_avg,
                "male_age": male_avg,
                "diff": diff,
                "female_count": female_cnt,
                "male_count": male_cnt
            })

        # Сортируем по абсолютной разнице и выводим топ-15
        results_df = pd.DataFrame(results).sort_values("diff", key=abs, ascending=False)

        print("Формат: [порода] → разница в пользу самок (+) или самцов (−)\n")
        for idx, row in results_df.head(15).iterrows():
            diff = row["diff"]
            sign = "🟢 +" if diff > 0 else "🔴 "
            if diff > 0:
                print(f"{sign}{abs(diff):.1f} лет | {row['breed']:30s} | "
                      f"самки ({row['female_age']:.1f} лет) vs самцы ({row['male_age']:.1f} лет) | "
                      f"N={row['female_count']}/{row['male_count']}")
            else:
                print(f"{sign}{abs(diff):.1f} лет | {row['breed']:30s} | "
                      f"самцы ({row['male_age']:.1f} лет) vs самки ({row['female_age']:.1f} лет) | "
                      f"N={row['male_count']}/{row['female_count']}")

        print(f"\nℹ️  Проанализировано {len(results_df)} пород с представителями обоих полов (≥5 особей на пол)")
        print("🟢 Зелёный = самки живут дольше | 🔴 Красный = самцы живут дольше")
    else:
        print("⚠️  Недостаточно данных для гендерного сравнения: нет пород с ≥5 особями каждого пола")

    # ============================================================================
    # 4. Ключевые выводы по полу
    # ============================================================================
    print("\n" + "="*70)
    print("💡 КЛЮЧЕВЫЕ ВЫВОДЫ ПО ПОЛУ")
    print("="*70)

    overall_gender = df_analysis.groupby(col_gender)["age"].agg(["mean", "count"]).round(2)
    if "самка" in overall_gender.index and "самец" in overall_gender.index:
        diff_overall = overall_gender.loc["самка", "mean"] - overall_gender.loc["самец", "mean"]
        print(f"• В среднем по всем породам: {'самки' if diff_overall > 0 else 'самцы'} "
              f"живут дольше на {abs(diff_overall):.1f} лет")
        print(f"  (самки: {overall_gender.loc['самка', 'mean']:.1f} лет, N={int(overall_gender.loc['самка', 'count'])} | "
              f"самцы: {overall_gender.loc['самец', 'mean']:.1f} лет, N={int(overall_gender.loc['самец', 'count'])})")
else:
    print("⚠️  Гендерный анализ невозможен: в данных отсутствует один из полов")
    print("   Доступные значения:", list(unique_genders))

# ============================================================================
# 5. Общий вывод по долгожителям
# ============================================================================
if not breed_stats.empty:
    top_breed = breed_stats.iloc[0]
    print(f"\n🏆 Самая долгоживущая порода: {top_breed.name} "
          f"(средний возраст {top_breed['avg_age']:.1f} лет, выборка N={int(top_breed['count'])})")

✅ Для анализа отобрано 1580 лошадей с корректными данными о возрасте

🏆 ТОП-10 ПОРОД ПО СРЕДНЕЙ ПРОДОЛЖИТЕЛЬНОСТИ ЖИЗНИ
 1. арабская лошадь                | Средний: 25.3 лет | Медиана: 27.0 | N= 19 | Диапазон: 8–40 лет
 2. Голштинская (порода лошадей)   | Средний: 23.8 лет | Медиана: 25.0 | N= 21 | Диапазон: 6–33 лет
 3. Французский сель               | Средний: 22.6 лет | Медиана: 23.0 | N=108 | Диапазон: 6–41 лет
 4. Ганноверская лошадь            | Средний: 22.5 лет | Медиана: 25.0 | N= 17 | Диапазон: 4–30 лет
 5. чистокровная верховая          | Средний: 19.1 лет | Медиана: 20.0 | N=1198 | Диапазон: 3–39 лет
 6. Першерон                       | Средний: 13.5 лет | Медиана: 13.0 | N= 85 | Диапазон: 2–33 лет

ℹ️  Всего проанализировано 6 пород с выборкой ≥15 особей

⚧️  ГЕНДЕРНАЯ КАРТА ВЫЖИВАЕМОСТИ
Доступные значения пола в данных: ['самец', 'самка']
Формат: [порода] → разница в пользу самок (+) или самцов (−)

🟢 +2.0 лет | Першерон                       | самки (14.7 лет) vs самцы 

## 📝 Summary

**Что мы сделали в этом ноутбуке (Week 2):**

- ✅ Клонировали ваш репозиторий GitHub в Colab
- ✅ Прочитали CSV-файл `data/horses.csv` с данными о лошадях из Викиданных
- ✅ Очистили данные: удалили URL Wikidata (`horse`) и переименовали столбцы (`*Label → короткие имена`)
- ✅ Проверили структуру данных (размер, типы столбцов, первые строки)
- ✅ Рассчитали возраст лошадей и выполнили валидацию:
  - отфильтровали аномалии (отрицательный возраст, возраст > 70 лет)
  - проанализировали распределение по годам рождения и смерти
  - изучили породовое разнообразие и гендерный состав

**Ключевые открытия:**
- 🏆 Выявили топ-10 пород-долгожителей по средней продолжительности жизни
- ⚧️ Построили гендерную карту выживаемости: сравнили долголетие самцов и самок внутри пород
- 🔍 Обнаружили исторические аномалии (например, лошади с годом рождения 1061)

Теперь у нас есть **очищенный, валидированный датасет** с информацией о кличках, годах жизни, поле и породах лошадей — готовый к углублённому анализу и визуализации.

В ноутбуке следующей недели мы продолжим исследование:
 -более сложного анализа
 - построения визуализации🐴📊